### json 불러와서 저장하기

In [1]:
import numpy as np
import pandas as pd
import json
import jpype
from bs4 import BeautifulSoup as bs

data_path = "data"

json_train=[]
for i in range(0,18):
    json_file = json.loads(open("%s/%d.json" % (data_path, i)).read())
    json_train += json_file["articles"]

### json에서 title 형태소 추출

In [2]:
from konlpy.tag import Mecab

mecab = Mecab()

labeled_train = []

for cnt, article in enumerate(json_train):
    
    if cnt % 10000 == 0:
        print cnt

    labeled_train.append({
            "istroll": article["is_troll"],
            "title_pos": ["%s_%s" % (first, second) for first, second in mecab.pos(article["title"])]
        })

labeled_train = pd.DataFrame.from_dict(labeled_train)
labeled_train

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000


,istroll,title_morphs
0,False,"[일간_NNG, 워스트_NNP, 마을_NNG, 회관_NNG, 입니다_VCP+EC]"
1,False,"[일간_NNG, 워스트_NNP, 일_NNG, 워_NNG, 입니다_VCP+EC]"
2,False,"[오_IC, !_SF]"
3,False,"[1_SN, 등_NNB]"
4,False,"[여기_NP, 가_JKS, 수용소_NNG, 냐_VCP+EF, ?_SF]"
5,False,"[여기_NP, 가_JKS, 일_NR, 워_IC, !_SF]"
6,False,"[지_MM, 젼_NNG]"
7,False,"[함수_NNG, 니_VCP+EC, +_SY, 빛돌_NNP, 이_JKS, 합동_NNG..."
8,False,"[캬_IC, ._SF, .._SY, 미루_VV, 고_EC, 미루_VV, 다_EC, ..."
9,False,"[일간_NNG, ㅋㅋㅋㅋㅋㅋㅋ워슼ㅋㅋㅋㅋㅋ틐ㅋㅋㅋㅋㅋㅋ_UNKNOWN]"


###bag of words

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

max_features = 1000

vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                            stop_words = None, max_features=max_features)

train_data_features = vectorizer.fit_transform((labeled_train["title_pos"].apply(lambda x:" ".join(x)))).toarray()

col = ["bow_title_%s" % data for data in vectorizer.get_feature_names()]
labeled_train = pd.concat([labeled_train, pd.DataFrame(train_data_features, columns = col, index=labeled_train.index)],axis=1)

labeled_train

,istroll,title_morphs,bow_title_0_sn,bow_title_100_sn,bow_title_10_sn,bow_title_11_sn,bow_title_12_sn,bow_title_15_sn,bow_title_16_sn,bow_title_18_sn,...,bow_title_회원_nng,bow_title_후_nng,bow_title_후기_nng,bow_title_후보_nng,bow_title_흔한_va,bow_title_힘들_va,bow_title_５_sn,bow_title_ｃ_sl,bow_title_ｏｍ_sl,bow_title_ｓｘｏ_sl
0,False,"[일간_NNG, 워스트_NNP, 마을_NNG, 회관_NNG, 입니다_VCP+EC]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,False,"[일간_NNG, 워스트_NNP, 일_NNG, 워_NNG, 입니다_VCP+EC]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,False,"[오_IC, !_SF]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,False,"[1_SN, 등_NNB]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,False,"[여기_NP, 가_JKS, 수용소_NNG, 냐_VCP+EF, ?_SF]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,False,"[여기_NP, 가_JKS, 일_NR, 워_IC, !_SF]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,False,"[지_MM, 젼_NNG]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,False,"[함수_NNG, 니_VCP+EC, +_SY, 빛돌_NNP, 이_JKS, 합동_NNG...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,False,"[캬_IC, ._SF, .._SY, 미루_VV, 고_EC, 미루_VV, 다_EC, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,False,"[일간_NNG, ㅋㅋㅋㅋㅋㅋㅋ워슼ㅋㅋㅋㅋㅋ틐ㅋㅋㅋㅋㅋㅋ_UNKNOWN]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### predictor, model 생성

In [4]:
from sklearn.ensemble import RandomForestClassifier

label = 'istroll'
pre = labeled_train.columns.drop(['title_pos', label])

model = RandomForestClassifier(n_estimators=10, n_jobs=6)

###cross validation

In [5]:
from sklearn import cross_validation

cv_value = 3

scores = cross_validation.cross_val_score(model, labeled_train[pre], labeled_train[label], cv=cv_value, scoring="roc_auc")
cv_result = scores.mean()

print(cv_result)

0.630053006037


##Result
  * title의 morphs를 bag of words로 변환 (feature : 1,000개) - 0.612752193303
  * title의 morphs를 bag of words로 변환 (feature : 1,000개) - 0.630053006037

In [6]:
import pickle

pickle.dump(labeled_train[pre], open("title_%d.p" % max_features, "wb"), protocol = pickle.HIGHEST_PROTOCOL)

,bow_title_017,bow_title_10,bow_title_100,bow_title_11,bow_title_12,bow_title_13,bow_title_14,bow_title_15,bow_title_16,bow_title_17,...,bow_title_확인,bow_title_활동,bow_title_회사,bow_title_회원,bow_title_후기,bow_title_후보,bow_title_흔한,bow_title_힘들,bow_title_ｏｍ,bow_title_ｓｘｏ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
